In [ ]:
%matplotlib inline

#Calculations
import numpy as np

#Plots
import matplotlib as mpl
from matplotlib import pyplot as plt

# Unit conversions and constant definitions
from astropy import constants as const
from astropy import units as u

# Optimizer:
from scipy.optimize import minimize

%pylab inline

# Make inline plots larger.
pylab.rcParams['figure.figsize'] = (14, 8)

# TeX rendering
mpl.rc('text', usetex = True)
mpl.rc('font', family = 'serif')

# Make axis tick labels larger
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15

#Generic object class for readability:
class Object:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
